# Pytorch图像分类模型转ONNX-三叶青分类

2024/5/1

## 安装配置环境


In [ ]:
%pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113  #安装 Pytorch
%pip install onnx -i https://pypi.tuna.tsinghua.edu.cn/simple  #安装 ONNX
%pip install onnxruntime -i https://pypi.tuna.tsinghua.edu.cn/simple #安装推理引擎 ONNX Runtime
%pip install numpy pandas matplotlib tqdm opencv-python pillow -i https://pypi.tuna.tsinghua.edu.cn/simple  #安装其它第三方工具包

## 导入工具包

In [1]:
import torch
import torchvision
from torch import nn

# 有 GPU就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device is', device)

device is cpu


# 从模型文件转化为权重文件

In [10]:
# 加载模型
model = torch.load(r"D:\model_bests_3.pth", map_location='cpu')
# 获取模型的权重
weights = model.state_dict()
#保存权重到文件中
torch.save(weights, 'model_bests_3.pth')

# GPU转为CPU（按需要进行）

In [11]:

import torch
import torchvision
from torch import nn
def get_mobilenet(device, model_name, num_classes=3):
        base_model_func = getattr(torchvision.models, model_name)   # 加载预训练的MobileNetV2模型
        base_model = base_model_func(pretrained=True)
        if model_name == "mobilenet_v2":
            num_features = base_model.classifier[-1].in_features
            classifier = nn.Sequential(    # 定义一个新的分类头
                nn.Linear(num_features, 256),
                nn.ReLU(),
                nn.Linear(256, num_classes)
            )
            base_model.classifier = classifier    # 替换原有的分类头
        else:
            num_features = base_model.features[-1].out_channels      # 获取最后一个卷积层的输出通道数量
            classifier = nn.Sequential(                # 定义一个新的分类头
                nn.Linear(num_features, 256),
                nn.ReLU(),
                nn.Linear(256, num_classes)
            )
            base_model.classifier = classifier  # 替换原有的分类头
        base_model = base_model.to(device)    # 将模型参数分配到指定设备
        for name, param in base_model.named_parameters():    # 冻结特征提取器的参数
            if 'classifier' not in name:  # 确保只冻结特征提取部分的参数
                param.requires_grad = False
        return base_model
def load_model_on_cpu(model_path, model_name):
    device = torch.device('cpu') # 创建模型
    model = get_mobilenet(device,model_name)
    state_dict = torch.load(model_path, map_location='cpu')  # 加载模型参数
    state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}   # 如果模型是在多GPU环境下训练的，模型参数的键会包含 'module.' 前缀，我们需要先去除这个前缀
    model.load_state_dict(state_dict)   # 加载参数到模型
    print(model)
    return model
# 使用方法
model_path = r"D:\SanYeQing_Project\wht_sanyeqing_image-Classification\【6】ONNX Runtime图像分类部署\model_bests_3.pth"
model = load_model_on_cpu(model_path,model_name="mobilenet_v3_large")
torch.save(model, 'model_cpu_3.pth')


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

## 加载训练得到的三叶青图像分类Pytorch模型文件

In [23]:
INPUT_DICT = r"D:\SanYeQing_Project\wht_sanyeqing_image-Classification\model_zheng_path\model_cpu_11.pth"  # 模型文件
OUT_ONNX = 'model_cpu_11.onnx' # 导出的onnx模型文件
x = torch.randn(1, 3, 224, 224)  # 定义一个随机的输入张量，用于导出模型。
input_names = ["input"] # 输入张量的名字
out_names = ["output"] # 输出张量的名字

model =torch.load(INPUT_DICT,map_location=device) # 加载自己训练的模型
print(model)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

## 评估模式

In [24]:
model = model.eval().to(device)
print(device)

cpu


## 输入Pytorch模型推理预测，获得n个类别的预测结果

In [25]:
# 当你有输入数据时
# x = x.to(device)  # 将输入数据移动到 CPU
output = model(x)
print(output.shape)

torch.Size([1, 11])


## Pytorch模型转ONNX模型

In [26]:
with torch.no_grad():
    torch.onnx.export(
                model,  #要转换的模型
                x,  #模型的任意一组输入
                OUT_ONNX,  #导出的ONNX文件名
                opset_version=11, #ONNX的算子集版本
                input_names=input_names, #输入Tensor的名称
                output_names=out_names, #输入Tensor的名称
    )

# onnxsim是一个用于优化和简化ONNX模型的工具，可以减小模型文件的大小并加速推理过程。
print('please run: python -m onnxsim test.onnx test_sim.onnx\n')


please run: python -m onnxsim test.onnx test_sim.onnx



## 验证onnx模型导出成功

In [16]:
import onnx

# 读取 ONNX 模型
onnx_model = onnx.load('model_cpu_97.18.onnx')
# 检查模型格式是否正确
onnx.checker.check_model(onnx_model)

print('ok,onnx模型载入成功')

ok,onnx模型载入成功


## 以可读的形式打印计算图

In [17]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph main_graph (
  %input[FLOAT, 1x3x224x224]
) initializers (
  %features.4.block.2.fc1.weight[FLOAT, 24x72x1x1]
  %features.4.block.2.fc1.bias[FLOAT, 24]
  %features.4.block.2.fc2.weight[FLOAT, 72x24x1x1]
  %features.4.block.2.fc2.bias[FLOAT, 72]
  %features.5.block.2.fc1.weight[FLOAT, 32x120x1x1]
  %features.5.block.2.fc1.bias[FLOAT, 32]
  %features.5.block.2.fc2.weight[FLOAT, 120x32x1x1]
  %features.5.block.2.fc2.bias[FLOAT, 120]
  %features.6.block.2.fc1.weight[FLOAT, 32x120x1x1]
  %features.6.block.2.fc1.bias[FLOAT, 32]
  %features.6.block.2.fc2.weight[FLOAT, 120x32x1x1]
  %features.6.block.2.fc2.bias[FLOAT, 120]
  %features.11.block.2.fc1.weight[FLOAT, 120x480x1x1]
  %features.11.block.2.fc1.bias[FLOAT, 120]
  %features.11.block.2.fc2.weight[FLOAT, 480x120x1x1]
  %features.11.block.2.fc2.bias[FLOAT, 480]
  %features.12.block.2.fc1.weight[FLOAT, 168x672x1x1]
  %features.12.block.2.fc1.bias[FLOAT, 168]
  %features.12.block.2.fc2.weight[FLOAT, 672x168x1x1]
  %features.12.block.2.

## 使用Netron可视化模型结构

Netron：https://netron.app

视频教程：https://www.bilibili.com/video/BV1TV4y1P7AP